# Hello World

## Simple invoke

In [ ]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",
    validate_model_on_init=True,
    temperature=0.8,
    num_predict=256,
    # other params ...
)

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming."),
    ("ai", "J'adore la programmation."),
    ("human", "Now in portuguese")

]
model.invoke(messages)

AIMessage(content='Agora em português: Adoro programação.', additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-20T22:19:14.015873942Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1049134742, 'load_duration': None, 'prompt_eval_count': 36, 'prompt_eval_duration': None, 'eval_count': 14, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'}, id='lc_run--b38da2ae-da16-49a3-81b8-1afa6aa0e0fb-0', usage_metadata={'input_tokens': 36, 'output_tokens': 14, 'total_tokens': 50})

## Streaming invoke

In [2]:
for chunk in model.stream("Return the words Hello World! separatedly."):
    print(chunk.text, end="")

Here are the words "Hello World!" separated individually:

**Hello**  
**World!**

## Asynchronous invoke

In [3]:
await model.ainvoke("Hello how are you!")

async for chunk in model.astream("Say hello world!"):
    print(chunk.content)

Hello world
!
 😊





## Batch invoke

In [4]:
messages = [("human", "Say hello world!"), ("human", "Say goodbye world!")]
await model.abatch(messages)

[AIMessage(content='Hello, world! 🌍', additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:00.883273419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 651696330, 'load_duration': None, 'prompt_eval_count': 10, 'prompt_eval_duration': None, 'eval_count': 7, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'}, id='lc_run--823925dc-9c18-4534-af3d-41f44c88dfb1-0', usage_metadata={'input_tokens': 10, 'output_tokens': 7, 'total_tokens': 17}),
 AIMessage(content='Goodbye, world! 👋  \nIf you have any more questions or need help in the future, I’m here for you. Take care! 😊', additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-01T23:39:01.207991262Z', 'done': True, 'done_reason': 'stop', 'total_duration': 963045650, 'load_duration': None, 'prompt_eval_count': 10, 'prompt_eval_duration': None, 'eval_count': 34, 'eval_duration': None, 'model

## Json format response

In [5]:
json_model = ChatOllama(model="deepseek-v3.1:671b-cloud", format="json")
res = json_model.invoke(
    "Return a query for the weather in a random location and time of day with two keys: location and time_of_day. "
    "Respond using JSON only."
)

print(res.content)

```json
{
  "location": "Kyoto",
  "time_of_day": "sunset"
}
```


## Tool use example

In [31]:
from typing import List
from langchain_ollama import ChatOllama
from langchain.tools import tool

@tool
def validate_user(user_id: int) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
    Returns:
        bool: whether the user is valid or not.
    """
    return user_id == 42

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",
    temperature=0,
).bind_tools([validate_user])

ans = model.invoke("Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX. Tell me if they are valid."
    "Tell me if the user is valid or not."
)

print(ans.tool_calls)
print(ans)

[{'name': 'validate_user', 'args': {'user_id': 123}, 'id': 'cab96f60-f45c-49be-95f7-11a6dde59f08', 'type': 'tool_call'}]
content="I'll validate user 123 for you using the available validation tool." additional_kwargs={} response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-20T22:39:25.229244596Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1037306492, 'load_duration': None, 'prompt_eval_count': 218, 'prompt_eval_duration': None, 'eval_count': 29, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'} id='lc_run--7cbcef5e-9929-4a55-8463-d8590e9281ea-0' tool_calls=[{'name': 'validate_user', 'args': {'user_id': 123}, 'id': 'cab96f60-f45c-49be-95f7-11a6dde59f08', 'type': 'tool_call'}] usage_metadata={'input_tokens': 218, 'output_tokens': 29, 'total_tokens': 247}


In [32]:
ans.tool_calls

[{'name': 'validate_user',
  'args': {'user_id': 123},
  'id': 'cab96f60-f45c-49be-95f7-11a6dde59f08',
  'type': 'tool_call'}]

In [33]:
print(ans.content)

I'll validate user 123 for you using the available validation tool.


In [34]:
ans

AIMessage(content="I'll validate user 123 for you using the available validation tool.", additional_kwargs={}, response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-20T22:39:25.229244596Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1037306492, 'load_duration': None, 'prompt_eval_count': 218, 'prompt_eval_duration': None, 'eval_count': 29, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'}, id='lc_run--7cbcef5e-9929-4a55-8463-d8590e9281ea-0', tool_calls=[{'name': 'validate_user', 'args': {'user_id': 123}, 'id': 'cab96f60-f45c-49be-95f7-11a6dde59f08', 'type': 'tool_call'}], usage_metadata={'input_tokens': 218, 'output_tokens': 29, 'total_tokens': 247})

In [8]:
from langchain.messages import AIMessage

if isinstance(ans, AIMessage) and ans.tool_calls:
    print(ans.tool_calls)

[{'name': 'validate_user', 'args': {'addresses': ['123 Fake St in Boston MA', '234 Pretend Boulevard in Houston TX'], 'user_id': 123}, 'id': '73462222-ecc7-4516-a3e4-51584ed43903', 'type': 'tool_call'}]


## Prompt templates example

In [38]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the context below.
    If the question can't be answered using the context, say "I don't know".

    Context: {context}

    Question: {question}

    Answer:""")

model = ChatOllama(
    model="deepseek-v3.1:671b-cloud",
)

prompt = template.invoke({
    "context": "The most recent advancements in AI technology have led to significant improvements in natural language processing.",
    "question": "What field has seen significant improvements due to recent advancements in AI?"
})

completion = model.invoke(prompt)

print(completion)

content='Answer: Natural language processing' additional_kwargs={} response_metadata={'model': 'deepseek-v3.1:671b-cloud', 'created_at': '2025-11-20T22:41:49.688801842Z', 'done': True, 'done_reason': 'stop', 'total_duration': 783003571, 'load_duration': None, 'prompt_eval_count': 71, 'prompt_eval_duration': None, 'eval_count': 6, 'eval_duration': None, 'model_name': 'deepseek-v3.1:671b-cloud', 'model_provider': 'ollama'} id='lc_run--bc2050a9-bb37-4c0b-aeed-a623bb67f98b-0' usage_metadata={'input_tokens': 71, 'output_tokens': 6, 'total_tokens': 77}


In [47]:
print(prompt.text)

Answer the question based on the context below.
    If the question can't be answered using the context, say "I don't know".

    Context: The most recent advancements in AI technology have led to significant improvements in natural language processing.

    Question: What field has seen significant improvements due to recent advancements in AI?

    Answer:


In [39]:
print(completion.content)

Answer: Natural language processing


## Chain example

In [40]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides answers with justifications. You must respond in JSON format using the specified keys."),
    ("human", "{question}"),
])

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0,
)

parser = JsonOutputParser(keys=["answer", "justification"])

chain = prompt | model | parser

response = chain.invoke(
    {"question": """What weight more, a pound of feathers or a pound of bricks? (use plain text)"""}
)

import json

print(json.dumps(response, indent=2))

{
  "answer": "They weigh the same: one pound each.",
  "justification": "A pound is a unit of weight; regardless of the material, a pound of feathers and a pound of bricks each have a mass of one pound, so their weights are equal."
}
